In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from PIL import Image
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from nltk.corpus import stopwords

import re
from sklearn.model_selection import train_test_split
import plotly.express as px
import warnings
import imblearn
from imblearn.over_sampling import SMOTE

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import GRU
from keras.layers import SimpleRNN
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.preprocessing import text
from keras.optimizers import Adam
# outdated: from keras.preprocessing import sequence 
from keras.utils.data_utils import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error

import squarify

import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

2023-09-09 14:40:28.643918: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from tensorflow.keras.models import load_model

loaded_model_lstm = load_model('final_lstm.h5')

2023-09-09 14:41:09.996542: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def clean_data(x):
    data_length=[]
    wnl=WordNetLemmatizer()
    cleaned_text=[]
    stop_words = set(stopwords.words('english')) # Load stop words
    mbti_types = [ "ISTJ", "ESFJ", "ISTJ","ISFP",
                  "ESTJ", "ESFP", "ENFP","ISTP",
                  "INFP", "ESTP", "INTP", "ENTP", 
                  "ENFJ", "INTJ", "ENTJ", "INFJ" ]
    mbti_types = [t.lower() for t in mbti_types]
    
    for sent in trump_tweets_df.Tweets:
        
        # Case Standardisation
        sent=sent.lower()
        
        # Remove URLs
        # source: Gabriel Giraldo-Wingler https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
        sent=re.sub('(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*',' ',sent)
        
        # Remove punctuations
        sent=re.sub('[^0-9a-z]',' ',sent)
        
        # Remove stop words
        sent = " ".join([word for word in sent.split() if word not in stop_words]) 
        
        for t in mbti_types:
            sent = re.sub(t, '', sent)
            
        # Lemmatize
        sent = wnl.lemmatize(sent) 
        
        data_length.append(len(sent.split())) #Split data, measure length of new filtered data
        
        cleaned_text.append(sent)
        
    return cleaned_text,data_length

In [5]:
class Lemmatizer(object):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()

    def __call__(self, sentence):
        return [self.lemmatizer.lemmatize(word) for word in sentence.split() if len(word)>2]

In [6]:
def preprocess_trump_tweets(trump_tweets):
    # Assuming trump_tweets is a list of tweets
    cleaned_trump_tweets = []
    
    for tweet in trump_tweets:
        # Apply the clean_data function to each tweet
        cleaned_tweet, _ = clean_data([tweet])
        cleaned_trump_tweets.append(cleaned_tweet[0])
    
    return cleaned_trump_tweets

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re


# Define a function to preprocess the tweets
def preprocess_tweet(tweet):
    # Remove URLs
    tweet = re.sub(r'http\S+', '', tweet)
    
    # Convert to lowercase
    tweet = tweet.lower()
    
    # Tokenize the tweet
    words = word_tokenize(tweet)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # Join words back into a sentence
    cleaned_tweet = ' '.join(words)
    
    return cleaned_tweet

In [10]:
trump_tweets_df = pd.read_csv('trumptweets.csv', encoding='ISO-8859-1')
trump_tweets_df.head()

,Tweets
0,Thank you to all of the television viewers tha...
1,Can you imagine if I had the small crowds that...
2,NATO commander agrees members should pay up vi...
3,"Wow, NATO's top commander just announced that ..."
4,"The ""Rust Belt"" was created by politicians lik..."


In [11]:
print(type(trump_tweets_df))


<class 'pandas.core.frame.DataFrame'>


In [12]:
# Preprocess Trump tweets using preprocess_trump_tweets function
trump_tweets_df['cleaned_tweet'] = trump_tweets_df['Tweets'].apply(preprocess_tweet)

In [15]:
import joblib
labelencoder = joblib.load('labelencoder_lstm.pkl')

# Tokenize and pad sequences
max_len = 40  # Adjust to match the model's input length
X_trump = tokenizer.texts_to_sequences(trump_tweets_df['cleaned_tweet'])
X_trump = pad_sequences(X_trump, padding='post', maxlen=max_len)

# Make a single prediction for all the tweets using the pre-trained model
predictions = loaded_model_lstm.predict(X_trump)

# Decode the prediction back to an MBTI type
predicted_mbti_enc = np.argmax(predictions, axis=1)  # Get the index of the highest probability
predicted_mbti = labelencoder.inverse_transform(predicted_mbti_enc)  # Inverse transform to get the MBTI label

# Print the predicted MBTI type
print("Predicted MBTI Type for Trump's Tweets:", predicted_mbti[0])

934/934 [==============================] - 21s 23ms/step
Predicted MBTI Type for Trump's Tweets: ENFP
